In [1]:
import pandas as pd
import numpy as np
from igraph import Graph

## Obtain Communities

In [2]:
# Function to obatain fastgreedy communities
def get_fastgreedy_communities(net):
    g = Graph.Load(net)
    dendrogram = g.community_fastgreedy(weights="weight")
    clusters = dendrogram.as_clustering()
    return g, clusters

In [3]:
# Build dataframe communities
def format_communities(g, clusters):
    
    def check_seller_role(row):
        if '_seller' in row['club_name']:
            row['club_name'] = row['club_name'].replace('_seller', '')
            row['role'] = 'seller'
        else:
            row['role'] = 'buyer'
        return row
    
    df_clusters = pd.DataFrame()
    for i, cluster in enumerate(clusters):
        cluster_idx = [i for c in cluster]
        club_name = [g.vs[c]["name"] for c in cluster]
        df_aux = pd.DataFrame({'cluster_idx': cluster_idx, 'club_name': club_name})
        df_clusters = pd.concat([df_clusters, df_aux])
    
    # Add league information
    df_league = pd.read_csv('resources_old_v2/club_league_v2.csv')
    df_league_sellers = df_league.copy()
    df_league_sellers['club_name'] = df_league_sellers['club_name'] + '_seller'
    df_clusters = pd.merge(df_clusters, df_league, how='left', on='club_name')
    df_clusters = pd.merge(df_clusters, df_league_sellers, how='left', on='club_name')
    df_clusters['league_name'] = df_clusters['league_name_x'].fillna(df_clusters['league_name_y'])
    df_clusters.drop(columns=['league_name_x', 'league_name_y'], inplace=True)
    
    df_clusters = df_clusters.apply(check_seller_role, axis=1)
    return df_clusters

In [4]:
# Get cluster's league composition
def get_league_composition(df_clusters):
    df_clusters_stats = df_clusters.groupby(['cluster_idx', 'role','league_name']).club_name.count().reset_index()
    df_clusters_stats_aux = df_clusters_stats.groupby(['cluster_idx', 'role']).club_name.sum().reset_index()
    df_clusters_stats = pd.merge(df_clusters_stats, df_clusters_stats_aux, how='inner', on=['cluster_idx', 'role'])
    df_clusters_stats.rename(columns={'club_name_x': 'count_league'}, inplace=True)
    df_clusters_stats['perc'] = 100*df_clusters_stats['count_league']/df_clusters_stats['club_name_y']
    df_clusters_stats.drop(columns='club_name_y', inplace=True)
    return df_clusters_stats

In [5]:
def filter_small_clusters(df_cluster_stats, min_clubs):
    df_cluster_stats_aux = df_cluster_stats[df_cluster_stats['role'] == 'buyer'].groupby('cluster_idx').count_league.sum().reset_index()
    df_cluster_stats_aux = df_cluster_stats_aux[df_cluster_stats_aux['count_league'] >= min_clubs]
    df_cluster_stats = pd.merge(df_cluster_stats, df_cluster_stats_aux[['cluster_idx']], how='inner', on='cluster_idx')
    return df_cluster_stats

In [6]:
# Do all function
def get_communities(net):
    g, clusters = get_fastgreedy_communities(net)
    df_clusters = format_communities(g, clusters)
    df_cluster_stats = get_league_composition(df_clusters)
    df_cluster_stats = filter_small_clusters(df_cluster_stats, 8)
    return df_clusters, df_cluster_stats

In [7]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib import cm

cdict = {'Premier League': 'royalblue', 'Primera Division': 'forestgreen', 'Serie A': 'orange', '1 Bundesliga': 'red',
         'Ligue 1': 'hotpink', 'Liga Nos': 'peachpuff', 'Eredivisie': 'gold', 'Premier Liga': 'cornflowerblue',
         'Austria': 'darkseagreen', 'Belgium': 'mediumorchid', 'Argentina': 'deepskyblue', 'Brazil': 'yellow',
         'Chile': 'lightsalmon', 'Croatia': 'pink', 'Denmark': 'lightcyan', 'Greece': 'aqua', 'Hungary': 'olive',
         'Mexico': 'yellowgreen', 'Norway': 'silver', 'Polonia': 'linen', 'Romania': 'navajowhite',
         'Scotland': 'lavender', 'Serbia': 'maroon', 'Sweden': 'cornsilk', 'Switzerland': 'honeydew',
         'Turkey': 'tan', 'Ukraine': 'mintcream', 'Uruguay': 'powderblue', 'Usa': 'dimgrey', 'Other': 'black'}

def get_pie_chart_data(df_cluster_stats, cluster, role):
    # Pie chart, where the slices will be ordered and plotted counter-clockwise:
    labels = df_cluster_stats[(df_cluster_stats['cluster_idx'] == cluster) &
                              (df_cluster_stats['role'] == role)]['league_name'].unique().tolist()
    sizes = df_cluster_stats[(df_cluster_stats['cluster_idx'] == cluster) &
                             (df_cluster_stats['role'] == role)]['perc'].tolist()
    
    # Obtain explode slice
    max_size_idx = sizes.index(max(sizes))
    aux = np.zeros(len(sizes))
    aux[max_size_idx] = 0.1
    explode = tuple(list(aux))
    
    return labels, sizes, explode

def get_pie_chart_plot(df_cluster, df_cluster_stats):
    cluster_list = df_cluster_stats['cluster_idx'].unique().tolist()
    num_clusters = len(cluster_list)
    fig, ax = plt.subplots(num_clusters, 2, figsize=(15, 75))
    
    j = 0
    for i in range(max(cluster_list)+1):
        if i not in cluster_list:
            continue
        # Get pie chart values
        labels_buyer, sizes_buyer, explode_buyer = get_pie_chart_data(df_cluster_stats, i, 'buyer')
        labels_seller, sizes_seller, explode_seller = get_pie_chart_data(df_cluster_stats, i, 'seller')
        # Get pie chart colors
        colors_buyer = [cdict[x] for x in labels_buyer]
        colors_seller = [cdict[x] for x in labels_seller]

        ax[j, 0].pie(sizes_buyer, explode=explode_buyer, labels=labels_buyer, autopct='%1.1f%%',
                     shadow=True, startangle=90, colors=colors_buyer)
        ax[j, 0].axis('equal')
        ax[j, 0].set_title('Cluster {} Buyers - League Proportion'.format(i), fontweight='bold')
        
        ax[j, 1].pie(sizes_seller, explode=explode_seller, labels=labels_seller, autopct='%1.1f%%',
                     shadow=True, startangle=90, colors=colors_seller)
        ax[j, 1].axis('equal')
        ax[j, 1].set_title('Cluster {} Sellers - League Proportion'.format(i), fontweight='bold')
        
        buyer_clubs = df_cluster[(df_cluster['cluster_idx'] == i) 
                          & (df_cluster['role'] == 'buyer')]['club_name'].unique().tolist()
        txt = ''
        for z, club in enumerate(buyer_clubs):
            if z % 10 == True:
                txt += club + '\n'
            else:
                txt += club + ', '
                
        ax[j, 0].text(0,-0.15,'Cluster Buyers {}: {}\n'.format(i, txt),transform=ax[j, 0].transAxes)
        
        j += 1
    
    plt.show()

## Super League Analysis

__GOAL:__ Use Super League teams (the european economical ellite of clubs) to show how the communities obtained completely vary if the weighting criteria is based on the total amount of transfers between clubs or the total money spend in these transactions

__Intuition:__ For the biggest clubs it is convenient to use their national leagues to do most of their signings (easier negotiations + similar culture + players that dream to play in their national leagues + usually less expensive transfers) but they trade among themselves (internatinal markets) when they want to buy more exclusive and expensive players. In the end the big amounts of money are always in the hands of the same clubs.

In [8]:
net_freq = 'nets/2015-2020/freq_net.net'
df_cluster_freq, df_cluster_stats_freq = get_communities(net_freq)

net_fee = 'nets/2015-2020/fee_net.net'
df_cluster_fee, df_cluster_stats_fee = get_communities(net_fee)

In [9]:
def get_preferred_league(df_cluster, df_cluster_stats, team, role):
    cluster = df_cluster[(df_cluster['role'] == role) 
                               & (df_cluster['club_name'] == team)]['cluster_idx'].item()
    if role == 'buyer':
        df_aux = df_cluster_stats[(df_cluster_stats['cluster_idx'] == cluster) & (df_cluster_stats['role'] == 'seller')]
    else:
        df_aux = df_cluster_stats[(df_cluster_stats['cluster_idx'] == cluster) & (df_cluster_stats['role'] == 'buyer')]
    
    perc = df_aux['perc'].max()
    max_league = df_aux[df_aux['perc'] == perc]['league_name'].iloc[0]
    
    return '{} - {}%'.format(max_league, np.round(perc,2))
    

In [10]:
def get_sl_teams_in_cluster(df_cluster_sl, team):
    roles = ['buyer', 'seller']
    num_sl = []
    num_sl_same_league = []
    for role in roles:
        cluster = df_cluster_sl[(df_cluster_sl['role'] == role) 
                                   & (df_cluster_sl['club_name'] == team)]['cluster_idx'].item()
        league = df_cluster_sl[df_cluster_sl['club_name'] == team]['league_name'].iloc[0]
        num_sl_aux = df_cluster_sl[(df_cluster_sl['cluster_idx'] == cluster) 
                                   & (df_cluster_sl['is_super_league'] == 1)]['club_name'].unique().tolist()
        num_sl_same_league_aux = df_cluster_sl[(df_cluster_sl['cluster_idx'] == cluster) 
                                   & (df_cluster_sl['is_super_league'] == 1) 
                                   & (df_cluster_sl['league_name'] == league)]['club_name'].unique().tolist()
        num_sl = num_sl + num_sl_aux
        num_sl_same_league = num_sl_same_league + num_sl_same_league_aux
    
    num_sl = len(set(num_sl)) - 1
    perc_sl_same_league = np.round(100*(len(set(num_sl_same_league)) - 1)/num_sl, 2)
    
    return num_sl, perc_sl_same_league

In [11]:
def mask_sl(row):
    if row['is_super_league'] > 0:
        row['is_super_league'] = 1
    else:
        pass
    return row

def sl_in_clusters(df_cluster_sl):
    # Clean small clusters
    df_cluster_sl_aux = df_cluster_sl[df_cluster_sl['role'] == 'buyer'].groupby('cluster_idx').club_name.count().reset_index()
    df_cluster_sl_aux = df_cluster_sl_aux[df_cluster_sl_aux['club_name'] >= 8]
    df_cluster_sl = pd.merge(df_cluster_sl, df_cluster_sl_aux[['cluster_idx']], how='inner', on='cluster_idx')
    
    # Get if super league team in cluster + cluster size
    df_sl_in_cluster = df_cluster_sl.groupby('cluster_idx').is_super_league.sum().reset_index()
    df_els_in_cluster = df_cluster_sl.groupby('cluster_idx').club_name.count().reset_index()
    
    df_sl_in_cluster = pd.merge(df_sl_in_cluster, df_els_in_cluster, how='inner', on='cluster_idx')
    df_sl_in_cluster = df_sl_in_cluster.apply(mask_sl, axis=1)
    
    all_clubs = df_sl_in_cluster['club_name'].sum()
    df_sl_in_cluster['club_name'] = df_sl_in_cluster['club_name']/all_clubs
    
    # Rename columns
    df_sl_in_cluster.rename(columns={'is_super_league': 'has_super_league', 'club_name': 'cluster_weight'}, inplace=True)
    
    return df_sl_in_cluster

def get_perc_clusters_w_sl(df_sl_in_cluster):
    df_sl_in_cluster['product'] = df_sl_in_cluster['has_super_league']*df_sl_in_cluster['cluster_weight']
    return df_sl_in_cluster['product'].sum()

In [12]:
super_league_teams = ['Real Madrid', 'FC Barcelona', 'Atlético Madrid',
                      'Manchester United', 'Manchester City', 'Chelsea FC',
                      'Liverpool FC', 'Arsenal FC', 'Tottenham Hotspur',
                      'Juventus FC', 'AC Milan', 'FC Internazionale',
                      'Bayern Munich', 'Borussia Dortmund', 'Paris Saint-Germain']

### Super League Teams - Community perspective per team

In [13]:
df_sl_teams = pd.DataFrame(columns=['club_name'], data=super_league_teams)
df_sl_teams['is_super_league'] = 1

df_cluster_sl_freq = pd.merge(df_cluster_freq, df_sl_teams, how='left', on='club_name').fillna(0)
df_cluster_sl_fee = pd.merge(df_cluster_fee, df_sl_teams, how='left', on='club_name').fillna(0)

In [14]:
data = {'Super League Team':[], 'Majority Buyer League - Freq': [], 'Majority Seller League - Freq': [],
        'Majority Buyer League - Fee': [], 'Majority Seller League - Fee': []}
data_2 = {'Super League Team':[], '#SL Teams - Freq': [], '%SL Teams National League - Freq': [],
        '#SL Teams - Fee': [], '%SL Teams National League - Fee': []}
for team in super_league_teams:
    max_buyer_league_freq = get_preferred_league(df_cluster_freq, df_cluster_stats_freq, team, 'buyer')
    max_seller_league_freq = get_preferred_league(df_cluster_freq, df_cluster_stats_freq, team, 'seller')
    max_buyer_league_fee = get_preferred_league(df_cluster_fee, df_cluster_stats_fee, team, 'buyer')
    max_seller_league_fee = get_preferred_league(df_cluster_fee, df_cluster_stats_fee, team, 'seller')
    data['Super League Team'].append(team)
    data['Majority Buyer League - Freq'].append(max_buyer_league_freq)
    data['Majority Seller League - Freq'].append(max_seller_league_freq)
    data['Majority Buyer League - Fee'].append(max_buyer_league_fee)
    data['Majority Seller League - Fee'].append(max_seller_league_fee)
    
    num_sl_freq, num_sl_same_league_freq = get_sl_teams_in_cluster(df_cluster_sl_freq, team)
    num_sl_fee, num_sl_same_league_fee = get_sl_teams_in_cluster(df_cluster_sl_fee, team)
    data_2['Super League Team'].append(team)
    data_2['#SL Teams - Freq'].append(num_sl_freq)
    data_2['%SL Teams National League - Freq'].append(num_sl_same_league_freq)
    data_2['#SL Teams - Fee'].append(num_sl_fee)
    data_2['%SL Teams National League - Fee'].append(num_sl_same_league_fee)
    
    

#### Majority league in the Super League Team's communities

In this section it is analysed the majority league present in the communities where the Super League Teams are placed either as buyer or sellers.

The analysis is split when the network is frequency or fee based

In [15]:
df_sl = pd.DataFrame(data=data)
df_sl

,Super League Team,Majority Buyer League - Freq,Majority Seller League - Freq,Majority Buyer League - Fee,Majority Seller League - Fee
0,Real Madrid,Primera Division - 58.49%,1 Bundesliga - 70.59%,Primera Division - 17.39%,1 Bundesliga - 33.33%
1,FC Barcelona,Primera Division - 58.49%,1 Bundesliga - 70.59%,Primera Division - 21.28%,Serie A - 33.33%
2,Atlético Madrid,Primera Division - 58.49%,Primera Division - 70.0%,Primera Division - 17.39%,Premier League - 25.0%
3,Manchester United,Primera Division - 58.49%,Premier League - 92.86%,Primera Division - 17.39%,Serie A - 33.33%
4,Manchester City,Primera Division - 58.49%,Premier League - 92.86%,Primera Division - 21.28%,Serie A - 33.33%
5,Chelsea FC,1 Bundesliga - 50.0%,Primera Division - 70.0%,Serie A - 28.57%,Primera Division - 26.92%
6,Liverpool FC,Premier League - 71.23%,Premier League - 92.86%,Premier League - 36.59%,Premier League - 25.0%
7,Arsenal FC,1 Bundesliga - 50.0%,1 Bundesliga - 70.59%,Primera Division - 21.28%,Premier League - 52.38%
8,Tottenham Hotspur,Premier League - 71.23%,Premier League - 92.86%,Serie A - 28.57%,Premier League - 25.0%
9,Juventus FC,Serie A - 68.97%,Serie A - 86.21%,Serie A - 28.57%,Primera Division - 26.92%


__Comment__

It can be seen how the majority league of the cluster of the teams of the super league varies depending on the usage of frequency of transfers as weight or the total fee of the transfers. In some of the cases, the majority league keeps being the same but the % of the winner is clearly lower, which examplifies how the fee communitites are much more heterogeneous. It is also interesting to see how the Italian and German teams do much more transfers with communities of their national league but how this completely changes when the transfer fee is taken into consideration. The same happens to Tottenham Hotspur in England. Which means that 33% of the super league teams change from national to international market depending on the weighting criteria.

In general it can be seen how the tendency is to majority go to the national league to do the more frequent transfers (with the exception of most of the english teams -except Liverpool and Tottenham-) while looking for international markets when having to do the most expensive transfers, which explain the variability between the frequency and fee communitites.

#### Presence of other SL Teams in the SL Team's communities

In this section we check per each SL team, the number of other super league teams that are in the same cluster. 

The analysis is split when the network is frequency or fee based

In [16]:
df_sl_2 = pd.DataFrame(data=data_2)
df_sl_2

,Super League Team,#SL Teams - Freq,%SL Teams National League - Freq,#SL Teams - Fee,%SL Teams National League - Fee
0,Real Madrid,9,22.22,10,10.00
1,FC Barcelona,9,22.22,10,10.00
2,Atlético Madrid,5,40.00,13,15.38
3,Manchester United,7,57.14,9,22.22
4,Manchester City,7,57.14,10,40.00
5,Chelsea FC,9,33.33,10,20.00
6,Liverpool FC,3,100.00,7,42.86
7,Arsenal FC,6,16.67,7,42.86
8,Tottenham Hotspur,3,100.00,12,33.33
9,Juventus FC,2,100.00,10,20.00


__Comment__

In this table it is analysed, per each of the teams, the number of Super League teams with whom they share cluster. The conclusions that can be extracted from this graph are similar to the previous table: there are some teams that clearly change their pattern depending on if the criteria is quantity based or fee based. A clear example of that are especially italian teams. All of them only share cluster with the other italian teams in the freq based communitites but this completely changes when checking it on a fee criteria. Again this examplifies how for the day to day transfers, which are not very expensive, they prefer to go to the national market, while when having to do the heavy investments they prefer to cluster with the elite of the teams of the continent. A similar patter happens with Bayern Munich, Borussia Dortmund and Paris Saint-Germain, which probably lack from important teams in their leagues to do this more expensive transfers, reason why they move to ellite communities to sign expensive players. Among the english teams, especially Tottenham and Liverpool are in a similar situation. On the other hand, the teams of Manchester or Arsenal present a more similar pattern in both types of communities, but in this case can be understood given the great amount of great teams in the Premier League, which may result in less necessity to go to international markets to invest money. Finally, there are 3 teams that clearly look like permanently installed in ellite clusters, regardless of the criteria that is used. These teams are Real Madrid, FC Barcelona and Chelsea FC. In the previous analysis it was seen how at least Barça and Madrid were in national communities, but in any case they are always together with some other elite teams, which also suggests how the Spanish League is an interesting market for the top european teams.

In general it can be concluded how most of the big teams don't usually trade together in terms of quantity of transfers but definetely do when the quantity analysed is of money: they are part of elitist economical communities that usually trade together when a lot of money is moved. In the end this looks like a loopy situation: these teams have more money, they can buy better players and when they trade this good players for a lot of money. only some other teams from the economical ellite can afford them, which results in the pattern observed in the data.

### Super League Teams - Aggregated Community perspective

In [17]:
df_sl_in_cluster_freq = sl_in_clusters(df_cluster_sl_freq)
df_sl_in_cluster_fee = sl_in_clusters(df_cluster_sl_fee)

In [18]:
df_sl_in_cluster_freq

,cluster_idx,has_super_league,cluster_weight
0,0.0,1.0,0.125378
1,1.0,1.0,0.132931
2,2.0,1.0,0.141994
3,3.0,1.0,0.173716
4,4.0,0.0,0.125378
5,5.0,1.0,0.131420
6,6.0,0.0,0.081571
7,7.0,0.0,0.087613


In [19]:
df_sl_in_cluster_fee

,cluster_idx,has_super_league,cluster_weight
0,0.0,1.0,0.109924
1,2.0,1.0,0.090076
2,3.0,0.0,0.250382
3,4.0,0.0,0.135878
4,5.0,1.0,0.102290
5,6.0,1.0,0.094656
6,7.0,0.0,0.048855
7,8.0,1.0,0.076336
8,9.0,0.0,0.091603


In [20]:
print('Percentage of clusters with SL teams - Freq: ', np.round(get_perc_clusters_w_sl(df_sl_in_cluster_freq), 2))
print('Percentage of clusters with SL teams - Fee: ', np.round(get_perc_clusters_w_sl(df_sl_in_cluster_fee), 2))

Percentage of clusters with SL teams - Freq:  0.71
Percentage of clusters with SL teams - Fee:  0.47


__Comment__

Interesting to see how when the frequencies networks, we obtain as many clusters as the total number of analysed leagues. On the other hand, when using fee, some extra small communities appear. It is also interesting to see the formation of ellite communities based on fee: it can be observed how the clusters weights are much less uniform (i.e. the number of teams in each cluster is pretty much the same when the analysis is based on frequency but is much more skewed if the fee is taken into consideration) and most importantly how the communities that have some Super League team in them are usually smaller, which reinforces this idea of having exclusive communities of the richest clubs. In that sense, the __Percentage of clusters with SL teams__ shown above is very clear: when using the frequency criteria more than 70% of the communities have at least one Super League Team. In this clustering scheme, the clusters are associated to a concrete league and usually the SL teams of that league use the national market to do most of their operations (even if they are not very expensive). On the other hand, __Percentage of clusters with SL teams__ goes down to 47% when using the fee criteria. This shows us how in terms of economic amount, not all the teams can be in the same community, because simply some of them cannot afford it. Thus, only less than half of the communities have SL teams on it. Which clearly draws an scenario where half of the teams are part of the "rich club" and the rest have to trade among themselves in the "poor club" party.